In [1]:
import pandas as pd
import requests
import time
import os
import sys

In [2]:
# --- AYARLAR ---
SOURCE_FILE = "human_data_final_3000.csv"  # Kaynak dosya (Başlıkları buradan alacağız)
OUTPUT_FILE = "ai_data_generated.csv"      # Üretilenlerin kaydedileceği dosya
MODEL_NAME = "llama3.1"                    # Kullandığın model
CHECKPOINT_INTERVAL = 20                   # Her 20 makalede bir kaydet

In [3]:
# --- 1. KAYNAK VERİYİ OKU ---
if not os.path.exists(SOURCE_FILE):
    print(f"Hata: {SOURCE_FILE} bulunamadı. Önce human verisini oluştur.")
    sys.exit()

df_source = pd.read_csv(SOURCE_FILE)
# Sadece başlıkları alalım, duplicate olmasın
all_titles = df_source['title'].unique().tolist()
total_titles = len(all_titles)

print(f"Kaynak dosya yüklendi. Toplam Başlık: {total_titles}")

Kaynak dosya yüklendi. Toplam Başlık: 3298


In [4]:
# --- 2. VARSA ESKİ ÜRETİLENLERİ YÜKLE (RESUME LOGIC) ---
generated_titles = []
ai_data_list = []

if os.path.exists(OUTPUT_FILE):
    try:
        df_existing = pd.read_csv(OUTPUT_FILE)
        # Hangi başlıklar zaten üretilmiş?
        generated_titles = df_existing['title'].tolist()
        # Mevcut veriyi hafızaya al ki üzerine ekleyelim
        ai_data_list = df_existing.to_dict('records')
        print(f"-> Önceki kayıttan devam ediliyor. {len(generated_titles)} makale zaten hazır.")
    except Exception as e:
        print(f"Mevcut dosya okunurken hata: {e}. Sıfırdan başlanıyor.")
else:
    print("-> Yeni üretim başlıyor.")

-> Önceki kayıttan devam ediliyor. 3100 makale zaten hazır.


In [5]:
# --- 3. FİLTRELEME (YAPILACAKLAR LİSTESİ) ---
# Sadece henüz yapılmamış olan başlıkları seç
titles_to_process = [t for t in all_titles if t not in generated_titles]
remaining_count = len(titles_to_process)

if remaining_count == 0:
    print("Tüm başlıklar için üretim tamamlanmış! İşlem gerekmiyor.")
    sys.exit()

print(f"-> Kalan üretilecek makale sayısı: {remaining_count}")
print("-" * 50)

-> Kalan üretilecek makale sayısı: 198
--------------------------------------------------


In [6]:
# --- 4. ÜRETİM DÖNGÜSÜ ---
url = "http://localhost:11434/api/generate"
counter = 0

try:
    for i, title in enumerate(titles_to_process):
        
        # Prompt Mühendisliği: Modelin akademik özet yazmasını sağlıyoruz
        # 'human' verisindeki özetlere benzer uzunlukta ve tonlamada olması önemli.
        prompt = f"""
        Act as an academic author. Write a research paper abstract for the title provided below.
        
        Title: "{title}"
        
        Constraints:
        1. Do NOT start with "Here is an abstract" or "In this paper". Start directly with the scientific text.
        2. Keep it between 5 to 10 sentences.
        3. Use formal, technical, and academic language appropriate for the topic (Physics/CS/Bio).
        4. Do not include citations or references.
        """
        
        payload = {
            "model": MODEL_NAME,
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.7, # Yaratıcılık dengesi
                "num_ctx": 4096     # Context penceresi
            }
        }
        
        try:
            # İsteği Gönder
            start_t = time.time()
            response = requests.post(url, json=payload)
            response.json() # JSON kontrolü
            
            if response.status_code == 200:
                ai_abstract = response.json()['response'].strip()
                
                # Listeye Ekle
                ai_data_list.append({
                    "title": title,
                    "summary": ai_abstract,
                    "label": "ai" # Etiketimiz AI
                })
                
                duration = time.time() - start_t
                counter += 1
                
                # Ekrana Bilgi Bas
                print(f"[{counter}/{remaining_count}] Üretildi ({duration:.1f}s): {title[:40]}...")

                # --- CHECKPOINT (KAYIT) ---
                # Belirli aralıklarla diske yaz ki elektrik kesilirse veri kaybolmasın
                if counter % CHECKPOINT_INTERVAL == 0:
                    pd.DataFrame(ai_data_list).to_csv(OUTPUT_FILE, index=False)
                    print(f"   >>> İlerleme Kaydedildi (Dosya Güncellendi).")

            else:
                print(f"Hata Kodu {response.status_code}: {title}")
                
        except Exception as e:
            print(f"Bağlantı/Model Hatası: {e}")
            # Hata olsa bile döngüyü kırma, bir sonrakini dene
            time.sleep(1)
            continue

except KeyboardInterrupt:
    # Kullanıcı CTRL+C yaparsa panik yapma, eldekileri kaydet ve çık.
    print("\n\n!!! İŞLEM KULLANICI TARAFINDAN DURDURULDU !!!")
    print("Mevcut veriler kaydediliyor...")
    pd.DataFrame(ai_data_list).to_csv(OUTPUT_FILE, index=False)
    print("Kayıt tamam. Çıkış yapılıyor.")
    sys.exit()

# --- BİTİŞ ---
# Döngü normal biterse son kez kaydet
pd.DataFrame(ai_data_list).to_csv(OUTPUT_FILE, index=False)
print("\n" + "="*50)
print(f"TÜM İŞLEMLER TAMAMLANDI!")
print(f"Dosya: {OUTPUT_FILE}")

[1/198] Üretildi (10.1s): Symmetry principles of gravitational per...
[2/198] Üretildi (5.8s): Analyzing GW231109_235456 and understand...
[3/198] Üretildi (5.5s): Avoiding PBH overproduction in inflation...
[4/198] Üretildi (5.8s): Potential of discovering $Ξ_{cc}^+$ in $...
[5/198] Üretildi (6.0s): Four loop renormalization of 3-quark ope...
[6/198] Üretildi (5.7s): Dynamical Dark Energy Meets Varying Elec...
[7/198] Üretildi (5.8s): How to Build an Empirical Speed Distribu...
[8/198] Üretildi (6.1s): Subleading Effects in Soft-Gluon Emissio...
[9/198] Üretildi (6.3s): Electroweak corrections to $gg\rightarro...
[10/198] Üretildi (6.0s): Five-loop beta function for gauge theori...
[11/198] Üretildi (6.4s): The Standard Model partial unification s...
[12/198] Üretildi (6.4s): Semileptonic $Λ_c \to Λ\ell ν_\ell$ Deca...
[13/198] Üretildi (6.0s): Gaussian Processes for Inferring Parton ...
[14/198] Üretildi (6.7s): Nonleptonic $Ω_{b}^{*}\rightarrowΩ_{c}^{...
[15/198] Üretildi (6.7s): Na